In [3]:
import pandas as pd
import numpy as np
import scipy
import random
import scipy.stats
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
symbols = ['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F']
def color_generator():
    color = '#'
    for i in range(6):
        index = 0
        for j in xrange(1000) :
            index = random.randint(0, 15)
        color += (symbols[index])
    return color

$\xi_{i}$ время меж- ду 𝑖-м моментом выходом из строя сервера и (𝑖+1)-м. 
Предполагается, что величины $\xi_{i}$ независимы в совокупности и имеют экспоненциальное распределение с парамет- ром $\lambda$ Т.е. $\xi_{i} \sim exp(\lambda)$
Для экспоненциального распределения, сопряженное - гамма с параметрами $(\alpha,\, \beta)$, а гиперпараметры апостериорного распределения будут  $\alpha+n,\, \beta+\sum_{i=1}^n x_i$

<img src="https://upload.wikimedia.org/wikipedia/commons/e/e6/Gamma_distribution_pdf.svg">

In [5]:
real_lambda = 0.213

In [6]:
break_time = pd.read_csv('/Users/semenfedotov/Desktop/MathStats/6.1.txt')

In [7]:
break_time = np.asarray(break_time['0.213'])

In [8]:
break_time

array([  5.473,   5.878,  13.314,  16.023,  17.636,  21.098,  24.429,
        26.512,  26.941,  27.613,  28.751,  37.977,  42.647,  55.828,
        59.729])

In [18]:
events = [(t, True) for t in break_time] + [(i, False) for i in range(61)] 
events.sort()

Выберем параметры априорного распределения таковыми, чтобы у плотности Гамма распределения не было горба(т.к. мы не знаем ничего, как распределена $\lambda$), и возьмем $\beta$, чтобы график плотности был гладким. Если брать параметр $\alpha$ > 1, то в окрестности некоторой точки образуется горб. Из рисунка выше, нам подходит модель с априорным распределением $\Gamma(1,2)$

Оценкой $\theta $ будет $\frac{\alpha + n - 1}{\beta + \sum {X_{i}}}$. Это в случае того, когда в качестве априорного распределения мы взяли $\Gamma(\alpha, \beta)$

Вот функция, с помощью которой мы будем оценивать параметр $\theta$

In [19]:
def evaluater(alpha, beta, sumka):
    return [(float(alpha) + float(i)) /(float(beta) + float(sumka[i])) for i in range(15)]

Запишем нашу выборку из экспоненциального распределения(разности между поломками)

In [21]:
selection = [break_time[0]]
for i in range(len(break_time) - 1):
    selection.append(break_time[i + 1] - break_time[i])
selection

[5.4729999999999999,
 0.40499999999999936,
 7.4360000000000008,
 2.7089999999999996,
 1.6129999999999995,
 3.4620000000000033,
 3.3309999999999995,
 2.0829999999999949,
 0.42900000000000205,
 0.67200000000000415,
 1.1379999999999946,
 9.2259999999999991,
 4.6700000000000017,
 13.181000000000004,
 3.9009999999999962]

Матожиданием будет $\lambda * (t - s)$

In [47]:
lambdas = evaluater(1, 2, np.cumsum(selection))

In [48]:
def conditional_expectation(t, s, lambda_parameter) :
    if s > 59.729 :
        return lambda_parameter * (t - s) + len(break_time)
    return lambda_parameter * (t - s) + np.where(break_time >= s)[0][0]

In [49]:
t = 60
times = range(0, 61)

Ну и выведем наши значения, либо Бреак, если была поломка, либо УМО, если поломки не было, УМО считаем с разными lambda, которые мы оценили

In [50]:
i = -1
print 'Time' + '\tE'
for event in events:
    if event[1]:
        print str(event[0]) + '\tbreak'
        i += 1
    else:
        if i != -1:
            print str(event[0]) + '\t' + str(conditional_expectation(t, event[0], lambdas[i]))
        else:
            print str(event[0]) + '\t' + '0'

Time	E
0	0
1	0
2	0
3	0
4	0
5	0
5.473	break
5.878	break
6	15.709063214
7	15.455191673
8	15.201320132
9	14.947448591
10	14.69357705
11	14.439705509
12	14.185833968
13	13.931962427
13.314	break
14	12.0113621523
15	11.8154629751
16	11.6195637978
16.023	break
17	13.5433612606
17.636	break
18	15.6946424934
19	15.4400081483
20	15.1853738032
21	14.9307394581
21.098	break
22	15.8709845008
23	15.6112217508
24	15.3514590008
24.429	break
25	16.270119944
26	16.0052593742
26.512	break
26.941	break
27	19.2622576967
27.613	break
28	20.8060649039
28.751	break
29	22.0890702741
30	21.7313583298
31	21.3736463855
32	21.0159344412
33	20.6582224968
34	20.3005105525
35	19.9427986082
36	19.5850866638
37	19.2273747195
37.977	break
38	18.6037971834
39	18.3036245841
40	18.0034519849
41	17.7032793856
42	17.4031067864
42.647	break
43	17.9499406455
44	17.6587676664
45	17.3675946872
46	17.0764217081
47	16.7852487289
48	16.4940757498
49	16.2029027706
50	15.9117297915
51	15.6205568123
52	15.3293838332
53	15.038210854
5